In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [5]:
!head {train_file_path}

'head'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [9]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [28. 58. 28. 19. 51.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 1 0 0 0]
fare                : [ 13.    153.462  26.55    7.896  77.958]
class               : [b'Second' b'First' b'First' b'Third' b'First']
deck                : [b'unknown' b'C' b'C' b'unknown' b'D']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [18.  4. 28. 28. 17.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 1 0 0 0]
fare                : [ 9.842 16.7    7.896 31.     7.125]
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'G' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [ 9. 15. 38. 44. 28.]
n_siblings_spouses  : [5 1 0 0 0]
class               : [b'Third' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'n' b'y' b'n' b'y']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [20. 28. 28.  7. 35.]
n_siblings_spouses  : [0. 0. 8. 0. 0.]
parch               : [0. 0. 2. 2. 0.]
fare                : [ 8.05 47.1  69.55 26.25 21.  ]


In [13]:
example_batch, labels_batch = next(iter(temp_dataset))

In [14]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [15]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[42.     1.     0.    26.   ]
 [56.     0.     0.    26.55 ]
 [40.     0.     0.     7.896]
 [28.     0.     0.    15.5  ]
 [28.     0.     0.    79.2  ]]

[1 0 0 0 1]


In [16]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [26. 18. 25. 22. 59.]
n_siblings_spouses  : [1 1 0 0 0]
parch               : [1 0 0 0 0]
fare                : [26.     6.496  7.65   7.229  7.25 ]
class               : [b'Second' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'F' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [17]:
example_batch, labels_batch = next(iter(temp_dataset))

In [18]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [19]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [20]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
class               : [b'First' b'Third' b'Third' b'Second' b'Third']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']
numeric             : [[35.     1.     0.    90.   ]
 [15.     1.     1.     7.229]
 [25.     0.     0.     7.05 ]
 [50.     0.     0.    10.5  ]
 [23.     0.     0.     7.854]]


In [21]:
example_batch, labels_batch = next(iter(packed_train_data))

In [22]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [23]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [24]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [25]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002A41D4D0C10>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [26]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[41.   ,  2.   ,  0.   , 14.108],
       [28.   ,  0.   ,  0.   , 39.6  ],
       [29.   ,  1.   ,  0.   , 26.   ],
       [44.   ,  0.   ,  1.   , 16.1  ],
       [16.   ,  0.   ,  0.   ,  8.05 ]], dtype=float32)>

In [27]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.909,  1.264, -0.479, -0.371],
       [-0.13 , -0.474, -0.479,  0.096],
       [-0.05 ,  0.395, -0.479, -0.154],
       [ 1.148, -0.474,  0.782, -0.335],
       [-1.089, -0.474, -0.479, -0.482]], dtype=float32)

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [32]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [33]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.909  1.264
 -0.479 -0.371  1.     0.   ]


In [34]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [35]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [36]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 3s 10ms/step - loss: 0.5556 - accuracy: 0.6756
Epoch 2/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3949 - accuracy: 0.8422
Epoch 3/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3898 - accuracy: 0.8317
Epoch 4/20
126/126 [==============================] - 1s 10ms/step - loss: 0.4103 - accuracy: 0.8353
Epoch 5/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3625 - accuracy: 0.8435
Epoch 6/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3905 - accuracy: 0.8211
Epoch 7/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3611 - accuracy: 0.8333
Epoch 8/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3704 - accuracy: 0.8318
Epoch 9/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3461 - accur

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.8523


Test Loss 0.4492773115634918, Test Accuracy 0.8522727489471436


In [38]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 12.46%  | Actual outcome:  DIED
Predicted survival: 9.42%  | Actual outcome:  DIED
Predicted survival: 55.78%  | Actual outcome:  DIED
Predicted survival: 85.99%  | Actual outcome:  SURVIVED
Predicted survival: 2.03%  | Actual outcome:  DIED
